
**PYWINTER WRF OUTPUT**

create input data with WRF output and modification

In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/My Drive/PWINTER/sample_data/
!pwd

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/PWINTER/sample_data
/content/drive/My Drive/PWINTER/sample_data


In [2]:
pip install netCDF4

In [3]:
pip install pywinter

In [4]:
import numpy as np
from netCDF4 import Dataset
import pywinter.winter as pyw


In [5]:
# Read info

ncfile = 'wout_d02_2018-11-24-06_00_00_00'
datefile = ncfile[9:-6]   # format YYYY-MM-DD_HH

print(datefile)

2018-11-24-06_00


In [13]:
nc = Dataset(ncfile,'r')

proj = getattr(nc, 'MAP_PROJ_CHAR')   # proj = 'Mercator' -> Geo1
dx = getattr(nc, 'DX')/1000 # m -> Km
dy = getattr(nc, 'DY')/1000 # m -> Km
tlat1 = getattr(nc, 'TRUELAT1')

print(dx)
print(dy)
print(tlat1)

9.0
9.0
6.194


In [26]:
lats = nc.variables['XLAT'][0,:,0]
lons = nc.variables['XLONG'][0,0,:]

# Read 3d info and modification 
uu3 = nc.variables['U'][0,:,:,:] + 1.0
ps3 = nc.variables['PB'][0,:,:,:] + nc.variables['P'][0,:,:,:]  # calculate pressure field

print(uu3.shape)
uu3 = uu3[:,:,:-1] # quick wind "destagger"
print(uu3.shape)

print(lats[0],' ',lats[-1])
print(lons[0],' ',lons[-1])

(49, 273, 274)
(49, 273, 273)
-4.817604   17.066734
-86.77437   -64.62622


In [27]:
# SWCORNER AT (lats[0],lons[0])

In [28]:

# PYWINTER

# pywinter geo-info
geoi = pyw.Geo1(lats[0],lons[0],dx,dy,tlat1)

# create pywinter variables
# V3D
pw_uu3 = pyw.V3d('UU',uu3)
pw_ps3 = pyw.V3d('PRESSURE',ps3)


#listing pywinter variables to include in intermediate files
listvar = [pw_uu3,pw_ps3]

# write intermediate files
pyw.cinter('WOUT',datefile,geoi,listvar)


WOUT:2018-11-24-06_00
